# Example A.1: Calculation of Howell Matrix Form

Given a matrix $A$ with entries from $\mathbb{Z}_N$, we calculate the Howell matrix form using a process similar to Gaussian elimination, but using a modified set of span-preserving row operations due to the fact that $\mathbb{Z}_N$ is a ring in general. The algorithm used in the XPF package is slightly different to algorithms presented elsewhere, but hopefully more intuitive.

Our objective is to calculate matrices $H, U$ and $K$ such that:

1. The matrix $H = UA$ is the Howell matrix form of $A$
2. The kernel of $A^T$ is spanned by $K$

### Ring Basics

In the ring $\mathbb{Z}_N$, not all elements have multiplicative inverses. Those that do have inverses are called **units**. For a given element $b$ we can always find a unit $u$ such that $ub = \text{GCD}_{\mathbb{Z}_N}(b,N)$ is a minimal representative. For a zero divisor $b$ we can always find an element $a$ which generates the annihilator of $b$ ie $ab = 0$ and $a'b =0 \implies a' = ac$ for some $c\in\mathbb{Z}_N$.

To produce an upper echelon form of matrices over $\mathbb{Z}_N$, we use the GCDEX function: $(g,s,t,u,v) = \text{Gcdex}(a,b,N)$. This has the following properties:

$$\begin{aligned}
g = sa + tb &=  \text{GCD}_{\mathbb{Z}_N}(a,b)\\
ua+vb &= 0\\
sv-tu&=1
\end{aligned}$$

### Span-Preserving Row Operations

The algorithm uses row operations to transform the matrix $A$ into the Howell form $H$. We keep a record of the row operations performed to determine $U$ and $K$. Allowed span-preserving row operations include:

1. Updating 2 rows by multiplying by a full rank $2 \times 2$ matrix: $\begin{pmatrix}A[r]\\A[j]\end{pmatrix} := B\begin{pmatrix}A[r]\\A[j]\end{pmatrix}$ where $\det(B) \ne 0$
2. Updating a row by multiplying it by a unit: $A[r] := uA[r]$
3. Appending the product of a row by any element $a\in\mathbb{Z}_N$ to the end of the matrix: $A.\text{append}(aA[r])$

### Transforming a Matrix into Howell Form using Row Operations

Given a matrix $A$ over $\mathbb{Z}_N$, the algorithm for calculating the Howell form of $A$ is as follows:

1. Set the row index $r=0$ and column index $c=0$
2. Determine if there exists some $A[j,c]> 0$ for $j \ge r$. If such a $j$ exists:
    - Ensure that $A[r,c] > 0$ by performing the SWAP row operation $\begin{pmatrix}A[r]\\A[j]\end{pmatrix} := \begin{pmatrix}0&1\\1&0\end{pmatrix}\begin{pmatrix}A[r]\\A[j]\end{pmatrix}$ if necessary 
    - Eliminate any entries below $A[r,c]$ by doing the following. For each $j > r$ with $A[j,c] > 0$, let $(g,s,t,u,v) = \text{Gcdex}(A[r,c],A[j,c],N)$. Perform the row operation $\begin{pmatrix}A[r]\\A[j]\end{pmatrix} := \begin{pmatrix}s&t\\u&v\end{pmatrix}\begin{pmatrix}A[r]\\A[j]\end{pmatrix}$. This ensures that $A[r,c] = g$ where $g = \text{GCD}_{\mathbb{Z}_N}(A[r,c],A[j,c])$ and $A[j,c] = 0$. This operation is span-preserving because $sv-tu = 1$.
    - Ensure that $A[r,c]$ is a minimal representative by finding a unit $u$ such that $uA[r,c]= \text{GCD}_{\mathbb{Z}_N}(A[r,c],N)$. Perform the row operation $A[r] := u A[r]$.
    - Ensure that $A[j,c] < A[r,c]$ for $j < r$ by determining $s = A[j,c] // A[r,c]$ (ie integer division). Perform the row operation $\begin{pmatrix}A[r]\\A[j]\end{pmatrix} := \begin{pmatrix}1&0\\-s&1\end{pmatrix}\begin{pmatrix}A[r]\\A[j]\end{pmatrix}$.
    - If $A[r,c]$ is a zero divisor, find the annihilator $a$ such that $a A[r,c]= 0$ and perform the row operation $A.\text{append}(aA[r])$ to append a new row. This ensures that the Howell property holds
    - Increment $r$ - halt if the number of rows in $A$ is equal to $r$
3. Increment $c$ - halt if the number of columns in $A$ is equal to $c$

### Determining the Transformation Matrix and Kernel

At the end of the above procedure, the first $k \le r$ rows of $A$ are non-zero and form the Howell matrix form. There may be a series of zero rows at the end. Let $H = A[:k]$

To find $U$ and $K$, we set $B = I_m$ where $m$ is the number of rows of $A$. Perform the row operations as determined above on $B$ and let $U = B[:k]$ and $K = B[k:]$. Then $UA = H$ and $KA = 0$. Because the row operations are span-preserving, $B = \begin{pmatrix}U\\K\end{pmatrix}$ is full rank and $K$ spans the kernel of $A^T$.

### Functionality of this Workbook
Below are a number of matrices with various precisions which illustrate the operation of the above algorithm. Specific examples can be selected by commenting/uncommenting the required lines. We also show how to check that the functions produce the expected results.

This code is made available subject to [GPL licensing](https://www.gnu.org/licenses/gpl-3.0.en.html). Readers who wish to modify the code can either download the [Github repository](https://github.com/m-webster/XPFpackage) or use online services such as [Binder](https://mybinder.org/) or [Colab](https://colab.research.google.com/).

In [23]:
import add_parent_dir
import numpy as np
from common import *
from NSpace import *

#######################################
####          Examples             ####
#######################################

## Examples from Storjohann and Mulders, Fast Algorithms for Linear Algebra Modulo N
## the following 4 matrices have the same Howell matrix form:
N = 12
A = ZMat([[8,5,5],[0,9,8],[0,0,10]])

# N = 12
# A = ZMat([[4,1,10],[0,0,5]])

# N = 12
# A = ZMat([[4,1,0],[0,0,1]])

# N = 12
# A = ZMat([[4,1,0],[0,3,0],[0,0,1]])

## Example from Storjohann's PhD Dissertation, Page 7 - both have the same Howell Form
# N = 16
# A = ZMat([[8,12,14,7]])

# N = 16
# A = ZMat([[8,12,14,7],[8,4,10,13]])

# ## Example from Storjohann's PhD Dissertation, Page 26
# N = 4
# A = ZMat([[2,3,2,2],[0,0,3,3],[2,3,2,2]])

## Typical example eg when calculating Canonical Generators
## When N is prime, same result as Gaussian Elimination
# N = 2
# A = [[0,0,0,1,0,0,0,0,0,0],
#      [0,0,0,1,0,0,1,1,1,0],
#      [0,0,0,1,0,1,0,1,0,1],
#      [0,0,0,1,0,1,1,0,1,1],
#      [0,0,1,0,1,0,0,0,1,1],
#      [0,0,1,0,1,0,1,1,0,1],
#      [0,0,1,0,1,1,0,1,1,0],
#      [0,0,1,0,1,1,1,0,0,0],
#      [0,1,0,0,0,0,0,1,0,0],
#      [0,1,0,0,0,0,1,0,1,0],
#      [0,1,0,0,0,1,0,0,0,1],
#      [0,1,0,0,0,1,1,1,1,1],
#      [1,0,0,0,1,0,0,1,1,1],
#      [1,0,0,0,1,0,1,0,0,1],
#      [1,0,0,0,1,1,0,0,1,0],
#      [1,0,0,0,1,1,1,1,0,0]]


print('The input matrix is A=')
print(ZmatPrint(A,N))
print('\nWe calculate the Howell Matrix form of A using the How() function.')
print('This results in the Howell matrix form H and a list of row operations required to convert A to H')
print('Howell Matrix form H=')
H, rowops = How(A,N)
print(ZmatPrint(H,N))
print('\nThe function CheckHow() verifies that each row of A is in the rowspan of H:')
print('CheckHow',CheckHow(A,H,N))
m = len(A)
print(f'\nThe number of rows in A is m={m}. We apply the row operations to a {m}x{m} identity matrix to find U and K:')
U, K = GetUK(A,H,rowops,N)
k = len(H)
print(f'\nThe number of rows in H is k={k}. U is the first k={k} rows of the transformed identity matrix: \nU=')
print(ZmatPrint(U,N))
print('\nK is formed from the remaining rows of the transformed identity matrix: \nK=')
print(ZmatPrint(K,N))
print("\nThe CheckUK() function verifies that: U A = H, K A = 0 and that matrix formed from the rows of U and K is full rank:")
print('CheckUK',CheckUK(A,H,U,K,N))


The input matrix is A=
 8  5  5
 0  9  8
 0  0 10

We calculate the Howell Matrix form of A using the How() function.
This results in the Howell matrix form H and a list of row operations required to convert A to H
Howell Matrix form H=
 4  1  0
 0  3  0
 0  0  1

The function CheckHow() verifies that each row of A is in the rowspan of H:
CheckHow True

The number of rows in A is m=3. We apply the row operations to a 3x3 identity matrix to find U and K:

The number of rows in H is k=3. U is the first k=3 rows of the transformed identity matrix: 
U=
 8  1  0
 0  7  4
 9  3  4

K is formed from the remaining rows of the transformed identity matrix: 
K=
 6  6  3
 0  4  4

The CheckUK() function verifies that: U A = H, K A = 0 and that matrix formed from the rows of U and K is full rank:
CheckUK True
